In [1]:
import pandas as pd
import os
import re
import requests
print(os.getcwd())

c:\Users\Joshualevy\Documents\ioCapture\adhoc_material\Reports\07-27


In [2]:
path_00_scraping = '../../../00_Scraping/'
path_01_database = '../../../01_Database_Construction/'

----------------------------------------------
### Journal Summary Statistics

The following code blocks generate code (and output tables) that help to answer the following questions:
- How many articles are there?
  - Across our 15 journals (we have to build ALJ into the workflow) there are 23,097 articles
- How many articles are there per journals?
  - See the tables generated by papers_database_summaries.R
  - This can be found as modified .tex file in scopus_econlit_matching_results_out.tex OR embedded in the report


In [3]:
articles_path = path_01_database + 'papers/fuzzy_matched_papers_df.csv'
articles_df = pd.read_csv(articles_path)
print(articles_df.columns)
print(len(articles_df))
articles_df.head(5)


Index(['doi_x', 'sc_title', 'sc_issn', 'sc_pub_name', 'sc_vol', 'sc_issue',
       'sc_page_range', 'sc_abstract_api_endpoint', 'sc_human_url',
       'sc_pub_date', 'sc_open_access_status', 'sc_query_used',
       'non_article_indicator_x', 'sc_title_upper', 'index', 'id', 'jel_desc',
       'jel_code', 'doi_y', 'title', 'volume', 'issue', 'date', 'pages',
       'issn', 'author', 'abstract', 'non_article_indicator_y', 'title_upper'],
      dtype='object')
23097


,doi_x,sc_title,sc_issn,sc_pub_name,sc_vol,sc_issue,sc_page_range,sc_abstract_api_endpoint,sc_human_url,sc_pub_date,...,title,volume,issue,date,pages,issn,author,abstract,non_article_indicator_y,title_upper
0,10.9767/BCREC.17.1.12473.22-31,Team-Specific Human Capital and Team Performan...,28282.0,American Economic Review,111.0,12,3923-3962,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/citedby.uri?part...,2021-12-01,...,Team-Specific Human Capital and Team Performan...,111.0,12,2021-12-01,3923-3962,28282.0,"['Chen, Yiqun']",This paper studies whether team members' past ...,0.0,TEAM-SPECIFIC HUMAN CAPITAL AND TEAM PERFORMAN...
1,10.9767/BCREC.17.1.12366.32-45,M Equilibrium: A Theory of Beliefs and Choices...,28282.0,American Economic Review,111.0,12,4002-4045,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/citedby.uri?part...,2021-12-01,...,M Equilibrium: A Theory of Beliefs and Choices...,111.0,12,2021-12-01,4002-4045,28282.0,"['Goeree, Jacob K.', 'Louis, Philippos']","We introduce a set-valued solution concept, M ...",0.0,M EQUILIBRIUM: A THEORY OF BELIEFS AND CHOICES...
2,10.9767/BCREC.17.1.12174.1-12,Sectoral Media Focus and Aggregate Fluctuations,28282.0,American Economic Review,111.0,12,3872-3922,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/citedby.uri?part...,2021-12-01,...,Sectoral Media Focus and Aggregate Fluctuations,111.0,12,2021-12-01,3872-3922,28282.0,"['Pitschner, Stefan', 'Chahrour, Ryan', 'Nimar...",We formalize the editorial role of news media ...,0.0,SECTORAL MEDIA FOCUS AND AGGREGATE FLUCTUATIONS
3,10.9767/BCREC.17.1.12412.53-64,Prep School for Poor Kids: The Long-Run Impact...,28282.0,American Economic Review,111.0,12,3963-4001,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/citedby.uri?part...,2021-12-01,...,Prep School for Poor Kids: The Long-Run Impact...,111.0,12,2021-12-01,3963-4001,28282.0,"['Sun, Shuqiao', 'Bailey, Martha J.', 'Timpe, ...",This paper evaluates the long-run effects of H...,0.0,PREP SCHOOL FOR POOR KIDS: THE LONG-RUN IMPACT...
4,10.9767/BCREC.17.1.12392.65-77,"Product Innovation, Product Diversification, a...",28282.0,American Economic Review,111.0,12,3795-3826,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/citedby.uri?part...,2021-12-01,...,"Product Innovation, Product Diversification, a...",111.0,12,2021-12-01,3795-3826,28282.0,"['Ohyama, Atsushi', 'Braguinsky, Serguey', 'Sy...",We explore how firms grow by adding products. ...,0.0,"PRODUCT INNOVATION, PRODUCT DIVERSIFICATION, A..."


----
### Authors Summary Statistics
- How many unique authors do we observe?
  - We have 52,461 author-article observations and 18,167 unique authors.
- How many articles are there per author
- Who are the top authors in each journal?


In [4]:
authors_path = path_01_database + 'authors/authors_df.csv'
authors_df = pd.read_csv(authors_path)
print(authors_df.columns)
print(len(authors_df))
authors_df.head(5)

Index(['doi', 'sc_title', 'sc_issn', 'sc_pub_name', 'sc_vol', 'sc_issue',
       'sc_page_range', 'sc_abstract_api_endpoint', 'sc_human_url',
       'sc_pub_date', 'sc_open_access_status', 'sc_query_used', 'sc_author_id',
       'sc_author_given_name', 'sc_author_last_name', 'sc_author_affil_id',
       'sc_author_affil_indexed', 'sc_grant_text', 'sc_funding_text',
       'sc_funding_agency', 'sc_author_indexed_name', 'sc_abstract_text'],
      dtype='object')
52461


,doi,sc_title,sc_issn,sc_pub_name,sc_vol,sc_issue,sc_page_range,sc_abstract_api_endpoint,sc_human_url,sc_pub_date,...,sc_author_id,sc_author_given_name,sc_author_last_name,sc_author_affil_id,sc_author_affil_indexed,sc_grant_text,sc_funding_text,sc_funding_agency,sc_author_indexed_name,sc_abstract_text
0,10.1257/aer.20201238,Team- Specific Human Capital and Team Performa...,28282,American Economic Review,111,12,3923-3962,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/citedby.uri?part...,2021-12-01,...,5.738864e+10,Yiqun,Chen,60027561,University of Illinois at Chicago,SCOPUS FAILURE,* University of Illinois at Chicago (email: yq...,SCOPUS FAILURE,Chen Y.,© 2021 American Economic Association. All righ...
1,10.1257/aer.20181801,Prep School for Poor Kids: The Long- Run Impac...,28282,American Economic Review,111,12,3963-4001,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/citedby.uri?part...,2021-12-01,...,1.463170e+10,Martha J.,Bailey,60027550,"Department of Economics, University of Califor...",SCOPUS FAILURE,"* Bailey: Department of Economics, University ...",SCOPUS FAILURE,Bailey M.J.,© 2021 American Economic Association. All righ...
2,10.1257/aer.20181801,Prep School for Poor Kids: The Long- Run Impac...,28282,American Economic Review,111,12,3963-4001,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/citedby.uri?part...,2021-12-01,...,5.738880e+10,Shuqiao,Sun,60112834,The World Bank,NaN,NaN,NaN,Sun S.,© 2021 American Economic Association. All righ...
3,10.1257/aer.20181801,Prep School for Poor Kids: The Long- Run Impac...,28282,American Economic Review,111,12,3963-4001,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/citedby.uri?part...,2021-12-01,...,5.718922e+10,Brenden,Timpe,60026306,"Department of Economics, University of Nebraska",NaN,NaN,NaN,Timpe B.,© 2021 American Economic Association. All righ...
4,10.9767/BCREC.17.1.12473.22-31,Team-Specific Human Capital and Team Performan...,28282,American Economic Review,111,12,3923-3962,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/citedby.uri?part...,2021-12-01,...,5.738864e+10,Yiqun,Chen,60027561,University of Illinois at Chicago,I gratefully acknowledge support from the Leon...,* University of Illinois at Chicago (email: yq...,SCOPUS FAILURE,Chen Y.,© 2021 American Economic Association. All righ...


In [6]:
unique_authors_count = len(authors_df.drop_duplicates(subset=['sc_author_id']))
print('Unique author count: {}'.format(unique_authors_count))

articles_per_author_df = authors_df[['sc_issn', 'sc_pub_name', 'sc_author_id', 'sc_author_given_name', 'sc_author_last_name']]

author_names_df = articles_per_author_df.drop_duplicates(subset='sc_author_id')[['sc_author_id', 'sc_author_given_name', 'sc_author_last_name']]
pub_names_df = articles_per_author_df.drop_duplicates(subset='sc_issn')[['sc_issn', 'sc_pub_name']]
# print(author_names_df)



articles_per_author = articles_per_author_df.groupby(by=['sc_author_id'])['sc_issn'].count()
articles_per_author = articles_per_author.to_frame().reset_index()
articles_per_author = articles_per_author.rename(columns={
    'sc_issn' : 'a_per_author_total'
})
print('Number of unique authors: {}'.format(len(articles_per_author)))

journal_articles_per_author =  articles_per_author_df.groupby(by=['sc_author_id', 'sc_issn'])['sc_pub_name'].count()
journal_articles_per_author = journal_articles_per_author.to_frame().reset_index()
journal_articles_per_author = journal_articles_per_author.rename(columns={
    'sc_pub_name' : 'a_per_author_per_journal'
})

print('Number of unique author-journals: {}'.format(len(journal_articles_per_author)))


per_author_df = pd.merge(journal_articles_per_author, articles_per_author, how='left', on='sc_author_id')
per_author_df = pd.merge(per_author_df, author_names_df, how='left', on='sc_author_id')
per_author_df = pd.merge(per_author_df, pub_names_df, how='left', on='sc_issn')

per_author_df.to_csv('tables/input/articles_per_author.csv', index=False, encoding='utf-8')

per_author_df


Unique author count: 18167
Number of unique authors: 18166
Number of unique author-journals: 31236


,sc_author_id,sc_issn,a_per_author_per_journal,a_per_author_total,sc_author_given_name,sc_author_last_name,sc_pub_name
0,6.503882e+09,0304405X,1,3,Alexei,Jiltsov,Journal of Financial Economics
1,6.503882e+09,221082,2,3,Alexei,Jiltsov,Journal of Finance
2,6.503885e+09,28282,1,1,Robert R.,Tchaidze,American Economic Review
3,6.503887e+09,0022166X,1,1,J. J.,Siegfied,Journal of Human Resources
4,6.503896e+09,129682,1,1,Michael,Mitzkewitz,Econometrica
...,...,...,...,...,...,...,...
31231,5.776081e+10,223808,1,1,Timothy N.,Cason,Journal of Political Economy
31232,5.776741e+10,8939454,1,1,Yan,Wang,Review of Financial Studies
31233,5.778440e+10,221082,1,1,Wei,Wang,Journal of Finance
31234,5.778668e+10,346527,1,1,Antonio,Ladrón-De-Guevara,Review of Economic Studies
